In [24]:
import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import Lasso
# from sklearn.linear_model import Ridge
# from sklearn.kernel_ridge import KernelRidge
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.cross_decomposition import PLSRegression
# from sklearn.cross_decomposition import PLSCanonical
from scipy.spatial.distance import cdist

In [25]:
# d: description, t: tag, f: 1000 feature, fi: 2048 intermediate features, bow: bag of word, w2v: word 2 vector
d_train_bow = np.load('preprocessing/bagOfWord_train_description.npy')
d_train_w2v = np.load('preprocessing/word2vec_train_description.npy')
t_train_bow = np.load('preprocessing/bagOfWord_train_tag.npy')
t_train_w2v = np.load('preprocessing/word2vec_train_tag.npy')
f_train = np.load('preprocessing/word2vec_train_1000feature.npy')
fi_train = np.load('preprocessing/word2vec_train_2048feature.npy')

print(d_train_bow.shape)
print(d_train_w2v.shape)
print(t_train_bow.shape)
print(t_train_w2v.shape)
print(f_train.shape)
print(fi_train.shape)

d_test_bow = np.load('preprocessing/bagOfWord_test_description.npy')
d_test_w2v = np.load('preprocessing/word2vec_test_description.npy')
t_test_bow = np.load('preprocessing/bagOfWord_test_tag.npy')
t_test_w2v = np.load('preprocessing/word2vec_test_tag.npy')
f_test = np.load('preprocessing/word2vec_test_1000feature.npy')
fi_test = np.load('preprocessing/word2vec_test_2048feature.npy')

print(d_test_bow.shape)
print(d_test_w2v.shape)
print(t_test_bow.shape)
print(t_test_w2v.shape)
print(f_test.shape)
print(fi_test.shape)

(10000, 7077)
(10000, 300)
(10000, 102)
(10000, 300)
(10000, 1000)
(10000, 2048)
(2000, 7077)
(2000, 300)
(2000, 102)
(2000, 300)
(2000, 1000)
(2000, 2048)


In [26]:
trainX = np.concatenate((t_train_bow, f_train, fi_train), axis=1)
print(trainX.shape)
trainY = d_train_bow
print(trainY.shape)

(10000, 3150)
(10000, 7077)


In [27]:
testX = np.concatenate((t_test_bow, f_test, fi_test), axis=1)
print(testX.shape)

(2000, 3150)


In [28]:
def output_prediction(testY, filename):
    distance = cdist(d_test_bow, testY, 'euclidean')
    f = open(filename, "w")
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i in range(2000):
        test_dist_idx = list(np.argsort(distance[i]))
        top_20 = test_dist_idx[:20]
        row = ["%d.jpg" % i for i in top_20]
        f.write("%d.txt,%s\n" % (i, " ".join(row)))
    f.close()
    print("Output written!")

In [29]:
##SVD
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=256, n_iter=7, random_state=42)
trainY=svd.fit_transform(trainY)
print(trainY.shape)

d_test_bow=svd.fit_transform(d_test_bow)
print(d_test_bow.shape)

(10000, 256)
(2000, 256)


In [31]:
## Nerual Network
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(512, activation=tf.nn.sigmoid),
    #keras.layers.Dense(3105, activation=tf.nn.sigmoid),
    keras.layers.Dense(256)
])

model.compile(optimizer='sgd', 
              loss='mean_squared_error')

In [33]:
model.fit(trainX, trainY, epochs=10)
testY=model.predict(testX)
output_prediction(testY, 'NN_submission.csv')

Epoch 1/10
10000/10000 [==============================] - 2s 200us/step - loss: 0.5513
Epoch 2/10
10000/10000 [==============================] - 2s 183us/step - loss: 0.4845
Epoch 3/10
10000/10000 [==============================] - 2s 183us/step - loss: 0.4670
Epoch 4/10
10000/10000 [==============================] - 2s 182us/step - loss: 0.4552
Epoch 5/10
10000/10000 [==============================] - 2s 184us/step - loss: 0.4464
Epoch 6/10
10000/10000 [==============================] - 2s 183us/step - loss: 0.4394
Epoch 7/10
10000/10000 [==============================] - 2s 182us/step - loss: 0.4336
Epoch 8/10
10000/10000 [==============================] - 2s 182us/step - loss: 0.4287
Epoch 9/10
10000/10000 [==============================] - 2s 182us/step - loss: 0.4243
Epoch 10/10
10000/10000 [==============================] - 2s 184us/step - loss: 0.4205
Output written!


In [34]:
# testY = LinearRegression(n_jobs=-1).fit(trainX, trainY).predict(testX)
# output_prediction(testY, 'linear_regression_submission.csv')

In [35]:
# testY = Lasso(n_jobs=-1).fit(trainX)